In [5]:
import numpy as np
import pandas as pd
import polars as pl
from sqlalchemy import create_engine

In [2]:
# First time pip installs
# pip install jupysql pyarrow psycopg

## Database Setup 

#### Downloading The Data

To download the data uncomment the code below

In [3]:
# pip install gdown
# !gdown 1NOhnBmfMpJqv3kJ-z9klC0JPmAc2g-6K

#### Loading Postgres

You may need to start postgres via `pg_ctl start -D pgdata` or any other method

In [4]:
%reload_ext sql

In [5]:
!psql postgresql://localhost/postgres -c 'DROP DATABASE IF EXISTS db'
!psql postgresql://localhost/postgres -c 'CREATE DATABASE db'

DROP DATABASE
CREATE DATABASE


#### Building the Database (ignore if using sample.csv)

We first sample 1,000,000 rows from the CSV file using pandas to create a smaller sampled dataset to construct the database with

In [ ]:
df = pl.read_csv('Covid-19_Ver2.csv', low_memory=False) # remove low_memory=False if its lagging

In [ ]:
df.head()

In [ ]:
df_sample = df.sample(n=1000000, seed=42)
df_sample.write_csv("sample.csv")

#### Loading from sample.csv

The cell below may take a long time to run

In [6]:
df_sample = pd.read_csv('sample.csv', low_memory=False)
engine = create_engine("postgresql://localhost/db")
df_sample.to_sql(name="covid", con=engine)
engine.dispose()

In [7]:
%sql postgresql://localhost/db

Connecting to 'postgresql://localhost/db'

In [8]:
%sqlcmd tables

Name
covid


In [9]:
%%sql
SELECT *
FROM covid
LIMIT 10;

Running query in 'postgresql://localhost/db'

10 rows affected.

index,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn,cdc_report_dtYear,cdc_report_dtMonth,cdc_report_dtWeek,cdc_report_dtDay,cdc_report_dtDayofweek,cdc_report_dtDayofyear,cdc_report_dtIs_month_end,cdc_report_dtIs_month_start,cdc_report_dtIs_quarter_end,cdc_report_dtIs_quarter_start,cdc_report_dtIs_year_end,cdc_report_dtIs_year_start,cdc_report_dtElapsed,pos_spec_dt_was_missing,onset_dt_was_missing,sex_was_missing,age_group_was_missing,Race and ethnicity (combined)_was_missing,pos_difference,onset_difference
0,2020-10-08,None,None,Laboratory-confirmed case,Female,80+ Years,"Black, Non-Hispanic",Yes,Missing,Yes,Unknown,2020,10,41,8,3,282,False,False,False,False,False,False,1602115200,True,True,False,False,False,None,None
1,2020-08-20,None,2020-08-09,Laboratory-confirmed case,Female,60 - 69 Years,"White, Non-Hispanic",No,No,Unknown,Missing,2020,8,34,20,3,233,False,False,False,False,False,False,1597881600,True,False,False,False,False,None,11.0
2,2020-07-11,None,2020-06-24,Laboratory-confirmed case,Female,20 - 29 Years,"White, Non-Hispanic",No,Missing,No,Missing,2020,7,28,11,5,193,False,False,False,False,False,False,1594425600,True,False,False,False,False,None,17.0
3,2020-10-14,None,2020-10-10,Laboratory-confirmed case,Male,40 - 49 Years,Hispanic/Latino,Missing,Missing,Missing,Missing,2020,10,42,14,2,288,False,False,False,False,False,False,1602633600,True,False,False,False,False,None,4.0
4,2020-08-04,None,None,Laboratory-confirmed case,Female,0 - 9 Years,Unknown,Missing,Missing,Missing,Missing,2020,8,32,4,1,217,False,False,False,False,False,False,1596499200,True,True,False,False,False,None,None
5,2020-08-15,None,2020-06-16,Laboratory-confirmed case,Female,50 - 59 Years,"Black, Non-Hispanic",No,Missing,No,Missing,2020,8,33,15,5,228,False,False,False,False,False,False,1597449600,True,False,False,False,False,None,60.0
6,2020-07-25,None,None,Laboratory-confirmed case,Male,40 - 49 Years,Unknown,No,Missing,No,Missing,2020,7,30,25,5,207,False,False,False,False,False,False,1595635200,True,True,False,False,False,None,None
7,2020-06-12,None,2020-06-12,Laboratory-confirmed case,Male,30 - 39 Years,Unknown,No,Missing,No,Missing,2020,6,24,12,4,164,False,False,False,False,False,False,1591920000,True,False,False,False,False,None,0.0
8,2020-05-15,None,None,Laboratory-confirmed case,Male,70 - 79 Years,"White, Non-Hispanic",Missing,Missing,Missing,Missing,2020,5,20,15,4,136,False,False,False,False,False,False,1589500800,True,True,False,False,False,None,None
9,2020-11-01,None,2020-10-22,Laboratory-confirmed case,Male,10 - 19 Years,Hispanic/Latino,No,Missing,Missing,Missing,2020,11,44,1,6,306,False,True,False,False,False,False,1604188800,True,False,False,False,False,None,10.0


In [10]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

## Query

For each query:
- Explain what task the query is trying to accomplish  
- Explain why this is a reasonable tasks for understanding or evaluating your chosen data system  
- An analysis of its performance. What could make it faster? What did you try to make it faster? For example, if you added an index, show the before and after.

In [11]:
%%sql
SELECT
  age_group,
  COUNT(*) AS male_lab_confirmed_cases
FROM covid
WHERE
  sex = 'Male'
  AND current_status = 'Laboratory-confirmed case'
GROUP BY age_group
ORDER BY age_group;


Running query in 'postgresql://localhost/db'

10 rows affected.

age_group,male_lab_confirmed_cases
0 - 9 Years,16665
10 - 19 Years,44026
20 - 29 Years,86177
30 - 39 Years,74471
40 - 49 Years,67577
50 - 59 Years,66398
60 - 69 Years,47997
70 - 79 Years,26304
80+ Years,16084
Unknown,1076


In [ ]:
# Trying to make it faster  

In [ ]:
# before and after speed 

In [ ]:
# Comparison with MongoDB Query